# Schema's in MongoDB

MongoDB database: collections met documenten

Collection: documenten "van dezelfde soort"

Document bepaalt **eigen schema**: namen en types van velden

*Hoe kunnen we afdwingen dat documenten in een collection **hetzelfde schema** (namen, types) hebben?*

In [1]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo

os.environ["PATH"]=os.environ["PATH"] + ":/usr/local/bin"
pd.set_option('max_colwidth',160)

userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
print("Database name: " + dbname)

Database name: eelco-demodb


Initialisatie van de database. Het resultaat moet 0 zijn.

In [2]:
from pymongo import MongoClient
print('Mongo version', pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client[dbname]

contacts = db.contacts
contacts.drop()
os.system('mongoimport -d ' + dbname + ' -c contacts adressen.json')

Mongo version 3.11.0


0

Toevoegen van een "vreemd" document

In [3]:
contacts.insert_one({"kleur": "groen", "prijs": 400, "beschrijving": "fiets"})

Zoeken op *type* van een veld:

In [4]:
list(contacts.find({"kleur": {"$type": "string"}}))

[{'_id': ObjectId('5fbbf996dac81d405cc77b4b'),
  'kleur': 'groen',
  'prijs': 400,
  'beschrijving': 'fiets'}]

## MongoDB collection-validator (schema)

* MongoDB query met namen en types

* met de mogelijkheid van alternatieven

* *minimale eis* aan documenten in collection (overige velden vrij)

In [6]:
contact_schema = {"name": {"$type": "string"},
                  "$or": [{"email": {"$type": "string"}},
                          {"tel": {"$type": "string"}}

                         ]}

In [7]:
list(contacts.find(contact_schema))

[{'_id': ObjectId('5fbbf98537a74f1b58525d4e'),
  'name': 'Hans de Boer',
  'email': 'hdb@example.com',
  'tel': '06-1290 8746'},
 {'_id': ObjectId('5fbbf98537a74f1b58525d4f'),
  'name': 'Anna Verschuur',
  'email': 'anna@hotmail.com',
  'address': {'street': 'Noorderkade 102',
   'city': 'Amsterdam',
   'postcode': '1000 AA'}},
 {'_id': ObjectId('5fbbf98537a74f1b58525d51'),
  'name': 'Gijs Bennekom',
  'email': 'gijsbkom@ziggo.nl',
  'address': {'street': 'Lijsterlaan 132', 'city': 'Rotterdam'}},
 {'_id': ObjectId('5fbbf98537a74f1b58525d52'),
  'name': 'Leontien de Bruin',
  'email': ['lhmdebruin@hotmail.com', 'leontien134@tiscalimail.nl'],
  'address': {'street': 'Tulpstraat 17', 'city': 'Amsterdam'}},
 {'_id': ObjectId('5fbbf98537a74f1b58525d53'),
  'name': 'Adrie Vierhuis',
  'email': 'a34huis@gmail.com',
  'address': {'street': 'Beukenweg 12',
   'city': 'Rotterdam',
   'postcode': '2001 BC'},
  'tel': '010-123 123 9'},
 {'_id': ObjectId('5fbbf98537a74f1b58525d54'),
  'name': 'Joop

Vinden van documenten die *niet* aan schema voldoen:

In [8]:
list(contacts.find({"$nor": [contact_schema]}))

[{'_id': ObjectId('5fbbf98537a74f1b58525d50'),
  'name': 'F.G. Schuitema',
  'address': {'street': 'Eikenlaan 23',
   'city': 'Amsterdam',
   'postcode': '1001 AB'}},
 {'_id': ObjectId('5fbbf996dac81d405cc77b4b'),
  'kleur': 'groen',
  'prijs': 400,
  'beschrijving': 'fiets'}]

Koppelen van schema (validator) aan collection:

In [9]:
db.command("collMod", "contacts", validator=contact_schema)

{'ok': 1.0}

toevoegen van niet-valide document:

In [10]:
contacts.insert_one({"naam": "Joke Baarsma", "email": "jokeb@friends.org"})

WriteError: Document failed validation, full error: {'index': 0, 'code': 121, 'errmsg': 'Document failed validation'}

Dat is handiger op te lossen met exception handling

In [12]:
try:
    contacts.insert_one({"name": "Joke Baarsma", "email": "jokeb@friends.org"})
except pymongo.errors.WriteError as s:
    print("Document not inserted: " + str(s))
else:
    print("insert OK")

insert OK


toevoegen van valide document:

In [ ]:
try:
    contacts.insert_one({"name": "Joke Baarsma", "email": "jokeb@friends.org"})
except pymongo.errors.WriteError as s:
    print("Document not inserted: " + str(s))
else:
    print("insert OK")

Oefenen met MongoDB schema's / validators